In [1]:
import pandas as pd
import numpy as np

from dsgd.DSClassifierMultiQ import DSClassifierMultiQ



from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Clasificadores
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/train.csv")
df = df.drop(columns=['index'])
print("Dataframe shape: ", df.shape)
df.head()

Dataframe shape:  (1400, 27)


,b2(N),b2(Y),b7(N),b7(Y),b6(N),b6(Y),b9(N),b9(Y),b5(N),b5(Y),...,b1(Y),b3(N),b3(Y),b12(N),b12(Y),b11(N),b11(Y),b13(N),b13(Y),label
0,0.007208,0.993486,0.007245,0.991205,0.047415,0.986543,0.003637,0.996469,0.003807,0.996930,...,0.995703,0.004138,0.996869,0.996687,0.001805,0.995002,0.003332,0.009329,0.985278,1
1,0.986265,0.012898,0.510913,0.630008,0.013700,0.995430,0.017154,0.991679,0.955430,0.027962,...,0.993046,0.995646,0.004970,0.995021,0.003918,0.982888,0.016396,0.994463,0.005285,1
2,0.019364,0.979410,0.123210,0.786819,0.995659,0.007027,0.006342,0.990831,0.006707,0.996069,...,0.931993,0.996315,0.003046,0.996893,0.002422,0.996145,0.003272,0.994930,0.005071,0
3,0.006707,0.996540,0.982360,0.012477,0.005348,0.997012,0.991996,0.004459,0.005064,0.995400,...,0.995125,0.008904,0.994745,0.015055,0.991430,0.019803,0.988209,0.988281,0.010148,1
4,0.991239,0.005303,0.019893,0.982599,0.022512,0.993508,0.008722,0.991827,0.005168,0.997139,...,0.997284,0.005189,0.994464,0.995104,0.002659,0.992518,0.008136,0.009482,0.991748,0


In [3]:
y = df['label']
X = df.drop(columns=['label'])    

In [4]:
y = y.to_numpy()
X = X.to_numpy()

In [5]:
DSCs = {}
time = {}
accuracys = {}
# 5 posibles learning rates
learning_rates = [0.0001, 0.001, 0.01, 0.1]
batch_sizes = [1000, 2000, 4000, 8000, 16000]
for lr in learning_rates:
    for bs in batch_sizes:
        DSC = DSClassifierMultiQ(2, min_iter=50, max_iter=1000, debug_mode=True, lr=lr, batch_size=bs,
                         lossfn="MSE", num_workers=1, min_dloss=1e-7, precompute_rules=True)
        DSCs[(lr, bs)] = DSC
        accuracys[(lr, bs)] = 0

In [6]:
for lr, bs in DSCs:
    DSC = DSCs[(lr, bs)]
    print("Training DSC with lr={} and bs={}".format(lr, bs))
    losses, epoch, dt = DSC.fit(X, y, add_single_rules=True,
                            single_rules_breaks=3, add_mult_rules=False,
                                column_names=df.columns[:-1], print_every_epochs=31, print_final_model=False)
    time[(lr, bs)] = dt

Training DSC with lr=0.0001 and bs=1000
Optimization started
Processing epoch	993	0.2059	
Training time: 2052.28s, epochs: 1000

Least training loss reached: 0.206
DSModelMultiQ(
  DS Classifier using 104 rules
  
  Rule 1: b2(N) < -0.070
  	C1: 0.124	C2: 0.076	Unc: 0.800
  
  Rule 2: -0.070 < b2(N) < 0.096
  	C1: 0.106	C2: 0.090	Unc: 0.804
  
  Rule 3: 0.096 < b2(N) < 0.262
  	C1: 0.157	C2: 0.041	Unc: 0.802
  
  Rule 4: b2(N) > 0.262
  	C1: 0.110	C2: 0.081	Unc: 0.810
  
  Rule 5: b2(Y) < 0.721
  	C1: 0.116	C2: 0.078	Unc: 0.806
  
  Rule 6: 0.721 < b2(Y) < 0.894
  	C1: 0.140	C2: 0.040	Unc: 0.820
  
  Rule 7: 0.894 < b2(Y) < 1.067
  	C1: 0.051	C2: 0.147	Unc: 0.802
  
  Rule 8: b2(Y) > 1.067
  	C1: 0.199	C2: 0.001	Unc: 0.800
  
  Rule 9: b7(N) < -0.021
  	C1: 0.122	C2: 0.078	Unc: 0.800
  
  Rule 10: -0.021 < b7(N) < 0.104
  	C1: 0.090	C2: 0.103	Unc: 0.808
  
  Rule 11: 0.104 < b7(N) < 0.229
  	C1: 0.192	C2: 0.000	Unc: 0.808
  
  Rule 12: b7(N) > 0.229
  	C1: 0.102	C2: 0.089	Unc: 0.809
  

In [7]:
#DSC.print_most_important_rules()

In [8]:
test = pd.read_csv("https://huggingface.co/datasets/furrutiav/sac_nllf/raw/main/val.csv")
test = test.drop(columns=['index'])
print("Dataframe shape: ", test.shape)
test.head()

Dataframe shape:  (200, 27)


,b2(N),b2(Y),b7(N),b7(Y),b6(N),b6(Y),b9(N),b9(Y),b5(N),b5(Y),...,b1(Y),b3(N),b3(Y),b12(N),b12(Y),b11(N),b11(Y),b13(N),b13(Y),label
0,0.004702,0.995775,0.023163,0.967443,0.003972,0.997503,0.989336,0.004162,0.005585,0.993807,...,0.987880,0.727084,0.313791,0.056407,0.959703,0.959625,0.040346,0.991056,0.005210,0
1,0.003834,0.997794,0.015057,0.992853,0.005001,0.997917,0.987899,0.007436,0.006093,0.994867,...,0.997831,0.004110,0.997463,0.064865,0.957161,0.008027,0.993875,0.003866,0.997420,1
2,0.009212,0.991151,0.022417,0.973221,0.982835,0.031924,0.005003,0.993315,0.006846,0.994212,...,0.139937,0.966394,0.020119,0.996717,0.002149,0.995235,0.004076,0.988816,0.013216,0
3,0.011463,0.990628,0.032560,0.975793,0.017151,0.991420,0.990053,0.002951,0.006522,0.991069,...,0.972626,0.029023,0.977822,0.025972,0.984690,0.239330,0.805682,0.991012,0.006145,1
4,0.009920,0.990553,0.014040,0.975804,0.670437,0.496262,0.004884,0.995518,0.006614,0.993191,...,0.737487,0.979781,0.010033,0.997109,0.002288,0.996718,0.002992,0.331006,0.350305,1


In [9]:
y_test = test['label']
X_test = test.drop(columns=['label'])
X_test = X_test.to_numpy()
y_test = y_test.to_numpy()

In [10]:
for lr, bs in DSCs:
    DSC = DSCs[(lr, bs)]
    y_pred = DSC.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracys[(lr, bs)] = acc
    print("Accuracy for lr={} and bs={}: {}".format(lr, bs, acc))
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

Accuracy for lr=0.0001 and bs=1000: 0.68
              precision    recall  f1-score   support

           0       0.72      0.70      0.71       112
           1       0.63      0.66      0.64        88

    accuracy                           0.68       200
   macro avg       0.68      0.68      0.68       200
weighted avg       0.68      0.68      0.68       200

[[78 34]
 [30 58]]
Accuracy for lr=0.0001 and bs=2000: 0.71
              precision    recall  f1-score   support

           0       0.75      0.71      0.73       112
           1       0.66      0.70      0.68        88

    accuracy                           0.71       200
   macro avg       0.71      0.71      0.71       200
weighted avg       0.71      0.71      0.71       200

[[80 32]
 [26 62]]
Accuracy for lr=0.0001 and bs=4000: 0.7
              precision    recall  f1-score   support

           0       0.74      0.71      0.73       112
           1       0.65      0.68      0.67        88

    accuracy          

: 